# SRIMAX Model

## Content
* Elements
* Data Preprocessing
* Model Identification
* Model Estimation
* Model Verification
* Model Use

Import required tools. 

In [ ]:
import warnings
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels as sm
import sklearn
import plotly
import matplotlib.pyplot as plt
%matplotlib inline

Get requiered config.

In [ ]:
# Set random seed for reproducibility
np.random.seed(0)

## Elements












### Definition
**Endogenous Variables**  
These are included in the model as the main time series data. The model aims to explain or predict these values based on historical data and other included factors.

**Exogenous Variables**  
These are included in the model as additional inputs that might affect the endogenous variable. 

### Sepecification
**SARIMAX(p,d,q)(P,D,Q)_E**   
**Seasonal Autoregressive Integrated Moving Averages Model with eXogenous Variables**  
Let $\{Z_t\}$ be a time series with seasonal period $E$ that describes a fenomenom that might be influenced by the variables $\{X_1\}$, $\{X_2\}$, ..., $\{X_n\}$. A seaosnal autorgegressive inegrated moving averages model with exogenoius variables is expresed as follows:
$$\phi(B)\Phi(B^{E})\nabla^d\nabla^D_E(Z_t) = \theta(B)\Theta(B^{E})a_t + \sum_{i = 1}^{n} X_i$$
Where:
* $\phi(B) = 1-\phi_1B - \phi_2B^{2} - ...- \phi_qB^{q}$ is a backshift polynomial of order $q$ that represents the  autoregressive part of the model. Thus, the current value of a time series is regressed on its own $q$ previous values. This is, $Z_{t}, Z_{t-1}, ..., Z_{t-q}$.
* $\theta(B) = 1-\theta_1B - \theta_2B^{2} - ...- \theta_pB^{p}$ is a backshift polynomial of order $p$ that represents the moving averages part of the  model. Thus, the current value of a time series can be expressed as a linear combination of $q$ past error terms. This is, the terms $a_{t}, a_{t-2}, ..., a_{t-p}$.
* $\Phi(B^E) = 1-\Phi_1B^E - \Phi_2B^{2E} - ...- \Phi_QB^{QE}$ is a seasonal backshift polynomial of order $Q$ that represents the seasonal autoregressive part of the model. Thus, the current value of a time series is regressed on its own $Q$ previous seasonal values. This is, $Z_{t-E}, Z_{t-2E}, ..., Z_{t-QE}$.
* $\Theta(B^E) = 1-\Theta_1B^E - \Theta_2B^{2E} - ...- \Theta_PB^{PE}$ is a seasonal backshift polynomial of order $P$ that represents the seasonal moving averages part of the  model. Thus, the current value of a time series can be expressed as a linear combination of $Q$ past seaosnal error terms. This is, the terms $a_{t-E}, a_{t-2E}, ..., a_{t-PE}$.
* \nabla_{E}^{D} is an seaosnal difference operator of order $D$.
* $a_t$ is a white noise porcess.

The main process $\{Z_t\}$ is referred as de endogenous variable while the processes $X_1, X_2,...,X_n$ are referred as exogenous variables or covariates. The idea behind the SARIMAX model is that these last variables may improve the performance of the model.
 
 Some examples of exogenous variables may include the following:
 * Macroeconomical variables
    * Interest Rates
    * Inflation Rates
    * Central Bank Reserves
    * GDP
    * Unemployment Rates
    * Exchange Rates
    * Consumer Confidence Index
    * Income Levels
    * Population Growth
    * Urbanization Trends
* Financial and Business Metrics
    * Stock Prices
    * Corporate Earnings
    * Operational Costs
* Market-Specific Factors
    * Price of Product
    * Product Availability
    * Competitor Pricing and Availability
    * Product Market Share
    * Marketing and Promotions
    * Consumer Trends and Preferences
    * Supply Chain Metrics
    * Supply and Demand Metrics
* External Events
    * Regulatory Changes
    * Geopolitical Events
    * Natural Disasters
* Commodity Prices
    * Energy Commodities
    * Metal Commodities
    * Agricultural Commodities
    * Livestock and Animal Products
    * Soft Commodities


### Forecastisting with Exogenous Variables
Let \{Z_t\} be a time series with seasonal period $E$ that describes a fenomenom that might be influenced by the variables $\{X_1\}$, $\{X_2\}$, ..., $\{X_n\}$. Supose you've got $N$ data points for the variable endogenous variable $Z_t$ and your forecast horizon is $K$ periods forward. Thus, in order to make such forecast, the exogenous variables $X_1,X_2,...,X_n$ must have $N+K$ data points. Therefore, the exogenous variables must be proyected $K$ periods forward. 

Moreover, take the following considerations:
* The exogenous variables, as well as the endogenous variable, must be mean-stabilized and variance-stabilized in order for them to be stationary. 

## Method
In order to get the job done, we'll proceed with the Box-Jenkins approach for time series models. 


## Data Preprocessing

### Load data

In [ ]:
# Load data
loading_path = r'C:\Users\MXKRDL01\OneDrive - Kellogg Company\Desktop\Workspace\AOS\AOS-Forecast\Data\BA_IZTAPALAPA_NORTE.csv'
df = pd.read_csv(loading_path)
df.info()

In [ ]:
# Select columns of interest
df = df[['ds', 'y', 'sales', 
        'inflation', 'consumer_confidence',  
        'ba_general_sold_units', 'ba_general_sales_value',
        'modern_channel_volume_share', 'modern_channel_avg_price_per_kilo']]
df.head()

In [ ]:
# Make sure dates are fine
df['ds'] = pd.to_datetime(df['ds'])
df.head()

In [ ]:
# Check for zeros
indices_to_erase = df[df['y'] == 0].index
indices_to_erase

In [ ]:
# Erase zeros
df = df.drop(indices_to_erase)
df = df.reset_index(drop=True)
df.head()

In [ ]:
#  Make a dataset till the last data point of endogenous variable.
df_present = df[df['ds'] <= '2024-07-01']

# Make a dataset that includes the proyected values for exogenous variables
df_furute = df

### Explore Data

In [ ]:
# View endogenous variable
df_present.plot(x = 'ds', y = 'y')

In [ ]:
## View endogenous variable with exogenous variables

# Import required tools
from sklearn.preprocessing import RobustScaler

# Create an auxiliary data frame
df_norm = df_present.copy()

# Normalize in order to avoid scale issues
df_norm = df_norm.drop('ds', axis=1)
scaler =  RobustScaler()
df_norm = scaler.fit_transform(df_norm)

# Put the data back into a pandas df
df_norm = pd.DataFrame(df_norm, columns=df.columns.drop('ds'))
df_norm['ds'] = df_present['ds']

# Plot
df_norm.plot(x='ds', y=df.columns.drop('ds'))

In [ ]:
# Get some basic statistics of engogenous variable as wel as exogenous variables
df_present.describe(include = [np.number])

In [ ]:
# Decompose the endogenous variable
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df_present['y'], model='additive', period = 52 ) 
result.plot()

In [ ]:
# Plot ACF and PACF of the endogenous variable
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


plot_acf(df_present['y'], lags=50)  
plt.title('ACF Plot')
plt.show()

plot_pacf(df_present['y'], lags=50)  
plt.title('PACF Plot')
plt.show()

### Prepare Data

In [ ]:
# Put endogenous variable into a pandas series object
endog_data = pd.Series(df_present['y'])
exog_data = df_present.drop(columns = ['y', 'ds'])

#### Seasonality Determination

**Seasonality determination by visual inspection**  
The goal is to look for patterns or repetitive cycles that recur at regular intervals. Check if there are obvious seasonal fluctuations.

In [ ]:
import matplotlib.dates as mdates

# View time series
plt.figure(figsize=(14, 8), dpi=100)  # Set the plot size to 14x8 inches and resolution to 100 DPI
plt.plot(df_present['ds'], df_present['y'])

# Set x-axis major locator to years and minor locator to months
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Time Series Plot')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

**Time Series Decomposition**  
The decomposition of time series assumes that the time series is composed of three main components: trend-cycle, which represents the long-term movement of the series; seasonality, which captures effects repeated annually with some consistency; and irregularity, which characterizes unpredictable and considered random movements. We'll focus in the seasonal part. 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Assuming df is your DataFrame and 'ds' is the date column, 'y' is the value column
df_E = df_present.copy()
df_E['ds'] = pd.to_datetime(df_E['ds'])
df_E.set_index('ds', inplace=True)

# Decompose the time series
result = seasonal_decompose(df_E['y'], model='additive', period=12)  # Adjust period as needed

plt.figure(figsize=(14, 8), dpi=100)
plt.plot(result.seasonal)
plt.title('Seasonality')
plt.show()

**ACP**  
Use the ACF to detect seasonal periods by identifying regular patterns or peaks. 
* In the ACF plot, significant peaks at regular intervals indicate the presence of seasonality. For example, if your data is monthly and you see peaks every 12 lags, this suggests a seasonal period of 12 months.
* The pattern of peaks will repeat with the seasonal period. For instance, if the ACF has a peak at lag 12, 24, 36, etc., this implies a seasonal period of 12.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

# Assuming df is your DataFrame and 'ds' is the date column, 'y' is the value column
df_E = df_present.copy()
df_E['ds'] = pd.to_datetime(df_E['ds'])
df_E.set_index('ds', inplace=True)

# Plot ACF
plt.figure(figsize=(14, 8), dpi=100)  # Set the plot size to 14x8 inches and resolution to 100 DPI
plot_acf(df_E['y'], lags=72)  # Adjust lags as needed to capture the seasonality

# Customize x-axis ticks
plt.xticks(range(0, 73, 12)) 

plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function (ACF)')
plt.grid(True)
plt.show()

#### Variance Stabilization for Endogneous Variable

In [ ]:
# Satabilize variance
from scipy.stats import boxcox
endog_data_bc, l = boxcox(endog_data)
print('Optimal lambda: ', l)

# Put endog_data_bc into a pandas series object
endog_data_bc = pd.Series(endog_data_bc)

#### Mean Stabilization for Endogenous Variable

In [ ]:
# Satbilize mean
endog_data_bc_diff = endog_data_bc.diff().dropna()

#### Dickey-Fuller Test for Endogenous Variable

In [ ]:
# Exectute Dickey-Fuller test for stationarity
from statsmodels.tsa.stattools import adfuller
result = adfuller(endog_data_bc_diff)
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

#### Normalizaton for Exogenous Variables

In [ ]:
# Import required tools
from sklearn.preprocessing import RobustScaler

# Initialize the StandardScaler
scaler = RobustScaler()

# Apply the StandardScaler to each column
exog_data_norm = pd.DataFrame(scaler.fit_transform(exog_data), columns=exog_data.columns)

# View the normalized data
print(exog_data_norm)

#### Dickey-Fuller Test for Exogenous Variables

In [ ]:
# Import required tools
from statsmodels.tsa.stattools import adfuller

# Exectute Dickey-Fuller test for stationarity for non-nomralized exogenous variables
for exog_var in ['sales', 'inflation', 'consumer_confidence', 'ba_general_sold_units', 'ba_general_sales_value', 'modern_channel_volume_share', 'modern_channel_avg_price_per_kilo']:
    result = adfuller(exog_data[exog_var])
    print('exogenous variable: ', exog_var)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])

## Model Identification
This section is about finding the $(p,d,q)(P,D,Q)_E$ parameters of a SARIMA model in order to get the best fit.

### Grid search (AIC)
he Akaike Information Criterion is a statistical measure used for model seletion. AIC is most useful when comparing models of the same data. It helps in comparing different models to determine which one best fits the data while accounting for model complexity. It is not as informative for absolute goodness-of-fit but is effective for relative comparisons. When you have several candidate models, calculate the AIC for each. The model with the lowest AIC is preferred. It is given by:

$$AIC = -2\ln(L) + 2k$$
Where:  
* $L$ is the log-likelihood function
* $k$ is the number of parameters


In [ ]:
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX


# Define the p, d, q, P, D, Q, s parameter ranges
p = d = q = range(0, 3)
P = D = Q = range(0, 3)
s = [4,8,12,16,20,24]  # Seasonal periods to consider

# Generate all different combinations of p, d, q, P, D, Q
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(P, D, Q, s))

# Perform grid search
best_aic = float("inf")
best_params = None

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                model = SARIMAX(endog = endog_data_bc,
                                exog = exog_data_norm,
                                order=param,
                                seasonal_order=(param_seasonal[0], param_seasonal[1], param_seasonal[2], param_seasonal[3]),
                                )
                results = model.fit()
                if results.aic < best_aic and param != (0,0,0):
                    best_aic = results.aic
                    best_params = (param, param_seasonal)
            except Exception as e:
                continue

print(f'Best SARIMA model: order={best_params[0]}, seasonal_order={best_params[1]} with AIC={best_aic}')

Thus, for some coefficients $\phi_1, \phi_2, \Theta_1, \Theta_2\in\mathbb{R}$ and the Box-Cox transformated time series $Z_t^{'}$,the SARIMAX model is:

$$(1- \phi_1B -\phi_2B^2)\nabla^1Z_t^{'} = (1-\Theta_1B^6-\Theta_2B^{12})a_t + CC_t + U_t + GS_t + GDP_t + IR_t + MXNUSD_t$$


## Model Estimation
In order to estimate the model coefficients we're going to use `statsmodels.tsa.statespace.sarimax.SARIMAX`. Thus, in order to get these, for a time series with edogenous variable previously box-cox transformed `endog_data_bc` stored on a `Series` object and exogenous data previously nomralized `exog_data_norm` stored on a `DataFrame` object  and parameters `(p,d,q)` and `(P,D,Q,s)` use the following code:
  
`from statsmodels.tsa.statespace.sarimax import SARIMAX`  
`model = SARIMAX(endog = endog_data,`  
`                 exog = exog_data,`  
`                 order = (p,d,q),`  
`                 seasonal_order = (P,D,Q,s))`  
`model = model.fit()`  
`model.summary()`  

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Best parameter combination
best_pdq = (2,0,1)
best_PDQs = (2,2,0,24)

# Estimate model
model = SARIMAX(endog = endog_data_bc,
                exog = exog_data_norm,
                order = best_pdq,
                seasonal_order = best_PDQs)
model = model.fit()
model.summary()

## Model Evaluation
In order to evaluate the model we're going to use two approaches. 
* Practical approach: we're interested on getting a nice fitted model with forecasting power. Thus, we're just going to check in sample metrics and forward cross-validation metrics
* Formal approach: We want to validate the assumptions on which the model is built. Thus, we'll validate each assumption of the model.

### In Sample Metrics

In [ ]:
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error
from scipy.stats import boxcox

# Best parameter combination
best_pdq = (2,0,1)
best_PDQs = (2,2,0,24)

# Get in-sample fitted values
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    best_model = SARIMAX(endog = endog_data_bc,
                         exog = exog_data_norm, 
                         order = best_pdq,
                         seasonal_order = best_PDQs)
    best_results = best_model.fit()
    fitted_values = best_results.fittedvalues

# Check for negative values and replace them with a small positive value. This is required for the Box-Cox transformation.
# We'll save the index and make it 1e-6 after the transformation.
negative_index = np.where(fitted_values < 0)[0]
fitted_values = np.where(fitted_values < 0, 1e-6, fitted_values)

# Inverse Box-Cox transformation
def inverse_boxcox(y_transformed, lmbda):
    if lmbda == 0:
        return np.exp(y_transformed)
    else:
        return np.exp(np.log(lmbda * y_transformed + 1) / lmbda)

# Apply inverse Box-Cox transformation to fitted values
ts, l = boxcox(df_present['y'])
fitted_values = inverse_boxcox(fitted_values, l)
actual_values = df_present['y']

# Substitute negative values that were converted to 1e-6 and Box-Cox inverse-transformed for 1e-6.
fitted_values[negative_index] = 1e-6

# Calculate the performance metrics
r2 = r2_score(actual_values, fitted_values)
mape = mean_absolute_percentage_error(actual_values, fitted_values)
mae = mean_absolute_error(actual_values, fitted_values)

print('r2: ', r2)
print('mape: ', mape)
print('mae: ', mae)

# Create the plot using Plotly
fig = go.Figure()

# Add actual values trace
fig.add_trace(go.Scatter(x = df_present['ds'], y = actual_values, mode='lines', name='Real'))

# Add fitted values trace
fig.add_trace(go.Scatter(x = df_present['ds'], y = fitted_values, mode='lines', name='Fitted', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='Real vs. Fitted Values',
    xaxis_title='Date',
    yaxis_title='Values',
    legend=dict(x=0, y=1)
)

# Show the plot
fig.show()

In [ ]:
# Temporary fix #
df_re = pd.DataFrame({'ds': df_present['ds'], 'Real': actual_values, 'Fitted': fitted_values})
df_re = df_re[df_re['ds']>= '2023-06-01']

actual_values = df_re['Real']
fitted_values = df_re['Fitted']

# Calculate the performance metrics
r2 = r2_score(actual_values, fitted_values)
mape = mean_absolute_percentage_error(actual_values, fitted_values)
mae = mean_absolute_error(actual_values, fitted_values)

print('r2: ', r2)
print('mape: ', mape)
print('mae: ', mae)

# Create the plot using Plotly
fig = go.Figure()

# Add actual values trace
fig.add_trace(go.Scatter(x = df_re['ds'], y = actual_values, mode='lines', name='Real'))

# Add fitted values trace
fig.add_trace(go.Scatter(x = df_re['ds'], y = fitted_values, mode='lines', name='Fitted', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='Real vs. Fitted Values',
    xaxis_title='Date',
    yaxis_title='Values',
    legend=dict(x=0, y=1)
)

# Show the plot
fig.show()


### Forward Cross-Validation Metrics

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from scipy.stats import boxcox

# Best parameter combination
best_pdq = (2,0,1)
best_PDQs = (2,2,0,24)

# Define the number of folds and the number of periods to forecast
k = 5
n_periods = 10

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=k, test_size=n_periods)

# Placeholder for the performance metrics
r2_scores = []
oos_r2_scores = []
mape_scores = []
mae_scores = []
iteration = 0

# Cross-validation
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for train_index, test_index in tscv.split(endog_data_bc):
        train_endog, test_endog = endog_data_bc.iloc[train_index], endog_data_bc.iloc[test_index]
        train_exog, test_exog = exog_data_norm.iloc[train_index], exog_data_norm.iloc[test_index]
        
        # Fit the SARIMA model
        model = SARIMAX(endog = train_endog, 
                        exog = train_exog,
                        order = best_pdq,
                        seasonal_order = best_PDQs)
        results = model.fit(disp=True)
        
        # Forecast
        forecast = results.get_forecast(steps = n_periods, exog = test_exog)
        forecast_values = forecast.predicted_mean

        # Inverse Box-Cox transformation
        def inverse_boxcox(y_transformed, lmbda):
            if lmbda == 0:
                return np.exp(y_transformed)
            else:
                return np.exp(np.log(lmbda * y_transformed + 1) / lmbda)
        
        # Apply inverse Box-Cox transformation to fitted values
        ts, l = boxcox(df_present['y'])
        forecasted_values = inverse_boxcox(forecast_values, l)
        actual_values = inverse_boxcox(test_endog, l)
        
        # Comparison between actual values and forecast
        df_comparison = pd.DataFrame({'y': actual_values.values.reshape(n_periods,), 'yhat': forecasted_values.values})
        
        # Calculate the performance metrics
        r2 = r2_score(actual_values, forecasted_values)
        def oos_r2_score(y_true, y_pred, y_dummy_pred):
            mse_pred = mean_squared_error(y_true, y_pred)
            mse_dummy = mean_squared_error(y_true, y_dummy_pred)
            oos_r2 = 1 - (mse_pred / mse_dummy)
            return oos_r2
        oos_r2 = oos_r2_score(actual_values, forecasted_values, np.full(len(actual_values), np.mean(train_endog)))
        mape = mean_absolute_percentage_error(actual_values, forecasted_values)
        mae = mean_absolute_error(actual_values, forecasted_values)

        # Print the performance metrics
        print('iteration: ', iteration)
        print('train size:', len(train_endog))
        print('test size: ', n_periods)
        print('r2: ', r2)
        print('oos_r2: ', oos_r2)
        print('mape: ', mape)
        print('mae: ', mae)
        print(df_comparison)

        r2_scores.append(r2)
        oos_r2_scores.append(oos_r2)
        mape_scores.append(mape)
        mae_scores.append(mae)

        iteration += 1

# Average R² and MAPE across all folds
average_r2 = np.mean(r2_scores)
average_oos_r2 = np.mean(oos_r2_scores)
average_mape = np.mean(mape_scores)
average_mae = np.mean(mae_scores)

print('GENERAL PERFORMANCE METRICS')
print('average r2: ', average_r2)
print('average oos_r2: ', average_oos_r2)
print('average mape: ', average_mape)
print('average mae: ', average_mae)

### Assumptions Verification

## Model Use
Now let's predict some future periods.

In [ ]:
# Set forecasting horizon
n_periods = 10

# Set exogenous variables proyected data
exog_data_future = df_furute.drop(columns = ['y', 'ds']).tail(n_periods)

# Normalice exogenpus vrariables proyecytion
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
exog_data_future_norm = pd.DataFrame(scaler.fit_transform(exog_data_future), columns=exog_data_future.columns)

# Best parameter combination
best_pdq = (2,1,0)
best_PDQs = (0,0,2,6)

# Fit the model with the best parameters
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    best_model = SARIMAX(endog = endog_data_bc,
                         exog = exog_data_norm, 
                         order = best_pdq, 
                         seasonal_order = best_PDQs)
    best_results = best_model.fit()
  
# Make forecast
forecast = best_results.get_forecast(steps = n_periods, exog = exog_data_future_norm)

# Extract predicted mean and confidence intervals
predicted_mean = forecast.predicted_mean
confidence_intervals = forecast.conf_int()  
  
# Combine into a single DataFrame
forecast_df = pd.DataFrame({
    'predicted_mean': predicted_mean,
    'lower_bound': confidence_intervals.iloc[:, 0],
    'upper_bound': confidence_intervals.iloc[:, 1]  
})  
  
# Inverse Box-Cox transformation`
def inverse_boxcox(y_transformed, lmbda):   
    if lmbda == 0:
        return np.exp(y_transformed)
    else:
        return np.exp(np.log(lmbda * y_transformed + 1) / lmbda)
ts, l = boxcox(df_present['y'])

# Apply inverse Box-Cox transformation to the forecast DataFrame  
forecast_df = forecast_df.map(lambda x: inverse_boxcox(x, l))  

forecast_df.head(n_periods)

## Pendientes
* Estacionariedad e invertibilidad estacional
* Ahondar en tranformación potencia Box-Cox
* Ahondar en número óptimo de diferencias estacionales y no estacionales
* Evaluar supuestos del modelo